In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [43]:
# File to Load (Remember to Change These)
fileordprod = "./data/brazilian-ecommerce/olist_order_items_dataset.csv"

# Read Purchasing File and store into Pandas data frame
ordprod = pd.read_csv(fileordprod)[['order_id', 'product_id']]
print(ordprod.shape)
ordprod.head()

(112650, 2)


,order_id,product_id
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089


In [44]:
# File to Load (Remember to Change These)
fileordcusttime = "./data/brazilian-ecommerce/olist_orders_dataset.csv"

# Read Purchasing File and store into Pandas data frame
ordcusttime= pd.read_csv(fileordcusttime)[['order_id', 'customer_id', 'order_purchase_timestamp']]
print(ordcusttime.shape)
ordcusttime.head()

(99441, 3)


,order_id,customer_id,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39


In [47]:
# File to Load (Remember to Change These)
filecustzip = "./data/brazilian-ecommerce/olist_customers_dataset.csv"

# Read Purchasing File and store into Pandas data frame
custzip = pd.read_csv(filecustzip)[['customer_id','customer_zip_code_prefix']]
print(custzip.shape)
custzip.head()

(99441, 2)


,customer_id,customer_zip_code_prefix
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409
1,18955e83d337fd6b2def6b18a428ac77,9790
2,4e7b3e00288586ebd08712fdd0374a03,1151
3,b2b6027bc5c5109e529d4dc6358b12c3,8775
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056


In [24]:
# File to Load (Remember to Change These)
fileprodcat = "./data/brazilian-ecommerce/olist_products_dataset.csv"

# Read Purchasing File and store into Pandas data frame
prodcat = pd.read_csv(fileprodcat)[['product_id', 'product_category_name']]
prodcat.head()


,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [25]:
# File to Load (Remember to Change These)
fileprodtransl = "./data/brazilian-ecommerce/product_category_name_translation.csv"

# Read Purchasing File and store into Pandas data frame
prodtransl = pd.read_csv(fileprodtransl)
prodtransl.head()


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [88]:
# File to Load (Remember to Change These)
filezipcoord = "./data/brazilian-ecommerce/olist_geolocation_dataset.csv"

# Read Purchasing File and store into Pandas data frame
zipcoord = pd.read_csv(filezipcoord)[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]
zipcoord = zipcoord.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix'}).drop_duplicates(subset = 'customer_zip_code_prefix')
print(zipcoord.shape)
zipcoord.head()



(19015, 3)


,customer_zip_code_prefix,geolocation_lat,geolocation_lng
0,1037,-23.545621,-46.639292
1,1046,-23.546081,-46.644820
3,1041,-23.544392,-46.639499
4,1035,-23.541578,-46.641607
5,1012,-23.547762,-46.635361


In [94]:
mergedf = pd.merge(ordcusttime, ordprod, on = 'order_id', how = 'right')
mergedf = pd.merge(mergedf, custzip, on = 'customer_id', how = 'left')
mergedf = pd.merge(mergedf, prodcat, on = 'product_id', how = 'left')
mergedf = pd.merge(mergedf, prodtransl, on = 'product_category_name', how = 'left')
mergedf = pd.merge(mergedf, zipcoord, on = 'customer_zip_code_prefix' , how = 'left')
mergedf = mergedf[['order_purchase_timestamp', 'product_category_name_english', 'geolocation_lat', 'geolocation_lng']]

renamecolumns = {
    'order_purchase_timestamp': 'time',
    'product_category_name_english': 'category',
    'geolocation_lat': 'lat',
    'geolocation_lng': 'lng'
}
mergedf = mergedf.rename(columns = renamecolumns)
print(mergedf.shape)
mergedf.head()

(112650, 4)


,time,category,lat,lng
0,2017-10-02 10:56:33,housewares,-23.574809,-46.587471
1,2018-07-24 20:41:37,perfumery,-12.169860,-44.988369
2,2018-08-08 08:38:49,auto,-16.746337,-48.514624
3,2017-11-18 19:28:06,pet_shop,-5.767733,-35.275467
4,2018-02-13 21:18:39,stationery,-23.675037,-46.524784
